In [1]:
%pip install tiktoken

     ---------------------------------------- 0.0/41.5 kB ? eta -:--:--
     --------- ------------------------------ 10.2/41.5 kB ? eta -:--:--
     ---------------------------- --------- 30.7/41.5 kB 325.1 kB/s eta 0:00:01
     -------------------------------------- 41.5/41.5 kB 333.1 kB/s eta 0:00:00
   ---------------------------------------- 0.0/894.9 kB ? eta -:--:--
   ---- ----------------------------------- 92.2/894.9 kB 1.7 MB/s eta 0:00:01
   ------ --------------------------------- 153.6/894.9 kB 1.8 MB/s eta 0:00:01
   ---------- ----------------------------- 225.3/894.9 kB 1.5 MB/s eta 0:00:01
   -------------- ------------------------- 317.4/894.9 kB 1.6 MB/s eta 0:00:01
   ----------------- ---------------------- 389.1/894.9 kB 1.7 MB/s eta 0:00:01
   ------------------- -------------------- 440.3/894.9 kB 1.6 MB/s eta 0:00:01
   ----------------------- ---------------- 522.2/894.9 kB 1.6 MB/s eta 0:00:01
   ------------------------- -------------- 563.2/894.9 kB 1.5 MB


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from importlib.metadata import version
import tiktoken
print("tiktoken version:", version("tiktoken"))

tiktoken version: 0.9.0


In [ ]:
# Use tiktoken BPE tokenizer
tokenizer = tiktoken.get_encoding("gpt2")

In [ ]:
# Encode
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
    "of someunknownPlace."
)
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [5]:
# Decode
strings = tokenizer.decode(integers)
print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


### Implementing a DataLoader with a sliding window

In [ ]:
# Read & encode text
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


In [7]:
# Remove some tokens for demonstration
enc_sample = enc_text[50:]

In [9]:
# Create input-target pairs
context_size = 4
x = enc_sample[:context_size]
y = enc_sample[1:context_size+1] # target is shifted
print(f"x: {x}")
print(f"y:      {y}")

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


In [10]:
# Create next-word prediction task
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(context, "---->", desired)

[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257


In [11]:
# Same but with text
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a
